<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLAMA3_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate -q

In [2]:
device = "cuda"

In [ ]:
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM

## v2 models
#openlm-research/open_llama_3b_v2
model_path = 'openlm-research/open_llama_3b_v2'
# model_path = 'openlm-research/open_llama_7b_v2'

## v1 models
# model_path = 'openlm-research/open_llama_3b'
# model_path = 'openlm-research/open_llama_7b'
# model_path = 'openlm-research/open_llama_13b'

tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)

model.todevice = device
#print(tokenizer.decode(generation_output[0]))

In [4]:
print(model.device)

cuda:0


In [5]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 3200, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (k_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (v_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (o_proj): Linear(in_features=3200, out_features=3200, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (up_proj): Linear(in_features=3200, out_features=8640, bias=False)
          (down_proj): Linear(in_features=8640, out_features=3200, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (l

In [73]:
prompt = 'Q: What is the largest animal?\nA:'
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=512, do_sample=True, temperature=0.7,
)

In [75]:
print(tokenizer.decode(generation_output[0]))

<s>Q: What is the largest animal?
A: The largest animal is the giraffe. It can grow up to 18.2 feet tall.
Q: What is the smallest animal?
A: The smallest animal is the dandelion. It can grow to be up to 1 inch tall.
Q: What is the tallest animal?
A: The tallest animal is the giraffe. It can grow up to 18.2 feet tall!
Q: What is the shortest animal?
A: The shortest animal is the dandelion. It can grow to be up to 1 inch tall!
Q: What is the largest bird?
A: The largest bird is the ostrich. It can grow up to 7 feet tall!
Q: What is the smallest bird?
A: The smallest bird is the hummingbird. It can grow as small as 2 inches tall!
Q: What is the longest bird?
A: The longest bird is the emu. It can grow up to 7 feet tall!
Q: What is the smallest bird?
A: The smallest bird is the hummingbird. It can grow as small as 2 inches tall!
Q: What is the longest bird?
A: The longest bird is the emu. It can grow up to 7 feet tall!
Q: What is the largest land mammal?
A: The largest land mammal is the g

In [78]:
prompt = 'Q: What is Canada?\nA:'

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to('cuda')

generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=512, do_sample=True, temperature=0.9,
)

In [79]:
print(tokenizer.decode(generation_output[0]))

<s>Q: What is Canada?
A: To Canadians, Canada is a nation. To Europeans, Canada is a country.. To us, Canada is our motherland.
Q: Where is Toronto?
A: It is the largest city of Canada. It is located in Ontario province.
Q: How many provinces and territories are there in Canada?
A: There are 13 provinces and 3 territories in Canada.
Q: Who is the first woman prime minister of Canada?
A: Edmund James W. Whitaker (1939–1983) led Liberal Party of Canada Party was first woman prime minister of Canada in 1963.
Q: What is the longest bridge in the world?
A: Golden Gate Bridge is the longest bridge in the world at 1.7 miles long with a span of 4,200 feet.
Q: What is the largest building in the world?
A: Burj Khalifa (The Building) is the tallest building in the world. It is located in Dubai
Q: How much salt does Canada contain?
A: It contains 244 million tonnes of salts.
Q: How many universities and colleges are there in Canada?
A: There are 84 universities and 55 colleges in Canada.
Q: What 